Сохранить один месяц данных из папки /datasets/marketplace в личную базу username.market_events, данные партиционировать по дате

Посчитать посуточные аггрегаты для этих данных по категориям в таблицу event_types_daily с колонками: event_date, category_id, event_type, event_count, distinct_customer_count. Счёт нужно производить отдельно по дням (в цикле по датам), целевую таблицу тоже разделить на партици

In [3]:
%spark.pyspark

market_events = spark.read.csv("/datasets/marketplace/2019_oct_parts.csv", header=True, inferSchema=True)
market_events.show(n= 5, truncate=False)

Задание 1

Сохранить один месяц данных из папки /datasets/marketplace/2019_oct_parts.csv в личную базу username.market_events, данные партиционировать по дате

In [4]:
%spark.pyspark

from pyspark.sql.functions import to_timestamp, col, to_date

spark.sql("create database if not exists kruglov")

(
market_events
.withColumn("event_time", to_timestamp(col("event_time"), 'yyyy-MM-dd HH:mm:ss'))
.withColumn("event_date", to_date(col("event_time")))
.write
.partitionBy("event_date")
.mode("overwrite")
.saveAsTable("kruglov.market_events")
)

spark.table("kruglov.market_events").show(5)

Задание 2

Посчитать посуточные аггрегаты для этих данных по категориям в таблицу event_types_daily с колонками: event_date, category_id, event_type, event_count, distinct_customer_count. Счёт нужно производить отдельно по дням (в цикле по датам), целевую таблицу тоже разделить на партиции.

In [5]:
%spark.pyspark


from pyspark.sql.functions import count, countDistinct


market_events = spark.table("kruglov.market_events")

for dates in market_events.select("event_date").distinct().collect():
    date = dates[0]
    event_types_daily = market_events.filter(market_events.event_date == date) \
        .groupBy("event_date", "category_id", "event_type") \
        .agg(count("event_type").alias("event_count"),
             countDistinct("user_id").alias("distinct_customer_count"))

    event_types_daily.write\
                     .partitionBy("event_date")\
                     .mode("append")\
                     .saveAsTable("kruglov.event_types_daily")

spark.table("kruglov.event_types_daily").show(5)

In [6]:
%spark.pyspark
